In [1]:
import pandas as pd

# READ DATASETS

In [2]:
folder = "./datasets/"
output_folder = "./datasets_models/"

### STATIONS DATASET


In [3]:
df_comarcas = pd.read_csv(folder + "t15902.csv", sep=";", header=0)

In [4]:
df_stations = pd.read_csv(folder + "DB_estaciones.txt", sep=";", header=0)

In [5]:
df_stations

,comarca,municipi,codiE,lat,lon,alt,data_alta,data_baixa,estat_actual
0,Alt Camp,Nulles,VY,41.25095,1.29863,240,18/09/2000,NaN,Operativa
1,Alt Camp,Vila-rodona,DA,41.30839,1.36220,278,23/01/1998,28/07/2005,Desmantellada
2,Alt Camp,Vila-rodona,DQ,41.30728,1.36259,287,29/07/2005,NaN,Operativa
3,Alt Empordà,Cabanes,U1,42.30648,2.95481,31,11/06/1991,NaN,Operativa
4,Alt Empordà,Castelló d'Empúries,W1,42.24726,3.09021,2,14/03/2000,NaN,Operativa
...,...,...,...,...,...,...,...,...,...
234,Vallès Oriental,La Roca del Vallès,KX,41.61865,2.36114,156,05/01/1996,NaN,Operativa
235,Vallès Oriental,Montmeló,CM,41.54911,2.24546,75,23/01/1996,22/03/2006,Desmantellada
236,Vallès Oriental,Parets del Vallès,XG,41.56734,2.22619,123,30/10/2008,NaN,Operativa
237,Vallès Oriental,Tagamanent,VX,41.74761,2.30291,1030,04/01/1996,NaN,Operativa


In [8]:

df_mesures = pd.read_csv(folder + "datos_precip.csv", header=0).drop(['Unnamed: 0'], axis=1)

In [10]:
df_mesures["data_lectura"] = df_mesures

,id,codi_estacio,codi_variable,data_lectura,valor_lectura
0,CA350101090000,CA,35,2009-01-01T00:00:00.000,0.0
1,CC350101090000,CC,35,2009-01-01T00:00:00.000,0.0
2,CD350101090000,CD,35,2009-01-01T00:00:00.000,0.0
3,CE350101090000,CE,35,2009-01-01T00:00:00.000,0.0
4,CG350101090000,CG,35,2009-01-01T00:00:00.000,0.0
...,...,...,...,...,...
22437293,Z7352905231200,Z7,35,2023-05-29T12:00:00.000,0.0
22437294,Z8352905231200,Z8,35,2023-05-29T12:00:00.000,0.7
22437295,Z9352905231200,Z9,35,2023-05-29T12:00:00.000,0.0
22437296,ZB352905231200,ZB,35,2023-05-29T12:00:00.000,0.0


In [22]:
df_mesures["valor_lectura"].max()

79.0

In [20]:
df_mesures[(df_mesures["codi_estacio"] == "VY") & (df_mesures.index == 11688)]["valor_lectura" ]

Series([], Name: valor_lectura, dtype: float64)

# PROCESS DATA

In [37]:
codiE_por_comarca = {}
for comarca, group in df_stations.groupby('comarca'):
    codiE_por_comarca[comarca] = list(group['codiE'].unique())

In [38]:
comarcas_bcn = ['Vallès Oriental', 'Alt Penedès', 'Baix Llobregat', 'Barcelonès', 'Garraf', 'Maresme', 'Vallès Occidental']
codiE_comarca = [codiE_por_comarca[i] for i in comarcas_bcn]

In [39]:
data_comarca = []
for i in comarcas_bcn:
    df_filtrado = df_mesures[df_mesures['codi_estacio'].isin(codiE_por_comarca[i])]
    data_comarca.append(df_filtrado.reset_index(drop=True))

In [40]:
for comarca in data_comarca:
    comarca.data_lectura = comarca.data_lectura.str.split('T').str[0]

In [41]:
media_por_dia_comarca = []
for comarca in data_comarca:
    comarca['data_lectura'] = pd.to_datetime(comarca['data_lectura'])
    media_por_dia_comarca.append(comarca.groupby([comarca['data_lectura'].dt.date, 'codi_variable'])['valor_lectura'].mean().reset_index())

In [42]:
nuevo_orden = ["comarca", "codi_variable", "data_lectura", "valor_lectura"]
for i in range(len(media_por_dia_comarca)):
    media_por_dia_comarca[i]["comarca"] = comarcas_bcn[i]
    media_por_dia_comarca[i] = media_por_dia_comarca[i][nuevo_orden].reset_index(drop=True)

In [43]:
df_concatenado = pd.concat(media_por_dia_comarca).reset_index(drop=True)
df_concatenado.to_csv(output_folder + "mean_temp_x_day.csv", index=False)

In [44]:
df_concatenado

,comarca,codi_variable,data_lectura,valor_lectura
0,Vallès Oriental,32,2009-01-01,8.059722
1,Vallès Oriental,32,2009-01-02,7.416667
2,Vallès Oriental,32,2009-01-03,5.665278
3,Vallès Oriental,32,2009-01-04,4.345833
4,Vallès Oriental,32,2009-01-05,3.138824
...,...,...,...,...
37095,Vallès Occidental,32,2023-07-05,21.832143
37096,Vallès Occidental,32,2023-07-06,22.760119
37097,Vallès Occidental,32,2023-07-07,23.723214
37098,Vallès Occidental,32,2023-07-08,25.869048
